In [1]:
import jieba
import re

def jieba_cut(_str):
    return ' '.join(jieba.lcut(_str))
def cnword_and_letter(_str):
    '''
    func: 文本清理，保留字符串中中文以及字母
    param: _str
        type: str
        detail
    return cleaned_str
        type: str
        detail: 经过strip去空格
    '''
    return "".join(re.findall("[\u4e00-\u9fa5a-zA-Z]", _str)).strip()

In [2]:
import pandas as pd

data_path = "./"

data = pd.read_csv(data_path + "train.csv")
data["question1"] = data["question1"].apply(cnword_and_letter)
data["question1"] = data["question1"].apply(jieba_cut)
data["question2"] = data["question2"].apply(cnword_and_letter)
data["question2"] = data["question2"].apply(jieba_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.619 seconds.
Prefix dict has been built succesfully.


In [3]:
import gensim
import numpy as np



def get_w2v(splited_corpus, w2v_size, min_count):
    '''
    func: 获取word2vec模型
    param: splited_corpus
        type: pd.Series
        detail: 应当为训练集中所有语料
    param: w2v_size
        type: int
        detail: w2v向量维度
    return: w2v_model
        type: gensim.models.Word2Vec
        detail: 训练的模型只可以使用其transform接口
    '''
    sentences = [x.split() for x in splited_corpus]
    model = gensim.models.Word2Vec(sentences, min_count=min_count, size=w2v_size)
    return model

def get_w2v_key_vev(w2v_model):
    vecs = []
    words = []
    for word in w2v_model.wv.vocab:
        vecs.append(w2v_model[word])
        words.append(word)
    return words, vecs

def get_x_index(x, words):
    res = []
    for inst in x:
        res.append(np.array([words.index(word) for word in inst.split() if word in words]))
    return res

def mean_len(list_2d):
    meanlen = 0
    for arr in list_2d:
        meanlen += len(arr)
    return meanlen / len(list_2d)

def ceil2(num):
    res = 2
    while res < num:
        res *= 2
    return res

def padding(data2d, max_len, pad_val):
    res = []
    for index, seq in enumerate(data2d):
        if(len(seq) < max_len):
            res.append(np.concatenate([seq, np.full([max_len - len(seq)], pad_val)]))
        else:
            res.append(seq[:max_len])
    return res

def concat_list_h(list1, list2):
    res = []
    for i, ele in enumerate(list1):
        res.append(np.concatenate([ele, list2[i]]))
    return res



corpus = list(data['question1']) + list(data['question2'])

w2v_model = get_w2v(corpus[0:20000], 20, min_count = 1)


In [4]:
w2v_model.most_similar(positive=['酸奶'])  

/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('红枣', 0.9968782663345337),
 ('南瓜', 0.9958581328392029),
 ('香蕉', 0.9936017394065857),
 ('蜂蜜', 0.9929537773132324),
 ('鸡蛋', 0.9928845167160034),
 ('三七', 0.9925090670585632),
 ('钙片', 0.9921731948852539),
 ('鸡汤', 0.9921368360519409),
 ('奶粉', 0.9897521734237671),
 ('蜂蜜水', 0.9890097975730896)]

In [7]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/imp.py", line 343, in load_dynamic
    return _l

In [6]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"

class Word2Vec():
    def __init__(self, w2v_size, model_type = "cbow", learning_rate = 0.025, learning_min = 1e-4, 
                 n_epochs = 1, c_of_cbow = 10, n_of_skgram = 10, min_frequency = 2):
        self.w2v_size = w2v_size
        self.model_type = model_type
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.min_frequency = min_frequency
        self.model_type = model_type = model_type.lower()
        if(model_type not in ["cbow", 'skip-gram']):
            self.model_type = model_type = "cbow"
            print("model type wrong set default as cbow")
        if(model_type == "cbow"):
            self.c_of_cbow = np.random.randint(1, c_of_cbow)
        elif(model_type == "skip-gram"):
            self.n_of_skgram = np.random.randint(1, n_of_skgram)
    
    def _frequency_dict(self, words):
        dic = {}
        for line in words:
            for word in line:
                if(word in dic):
                    dic[word] += 1
                else:
                    dic[word] = 1
        return dic
    
    def _onehot_encode(self, index, max_len):
        res = np.zeros([max_len])
        if(index >= 0):
            res[index] = 1
        return res
    
    def _reduce(self, frequency_dict):
        if(self.min_frequency > 1):
            del_words = []
            for word in self.frequency_dict:
                if(self.frequency_dict[word] < self.min_frequency):
                    del_words.append(word)
            for dword in del_words:
                del self.frequency_dict[dword]
        return frequency_dict
   
    def fit(self, splited_corpus):
        self.words = [x.lower().split() for x in splited_corpus]
        self.frequency_dict = self._frequency_dict(self.words)
        self.frequency_dict = self._reduce(self.frequency_dict)
        self.vocab = list(self.frequency_dict.keys())
        self.vocab_index = {word : i for (i, word) in enumerate(self.vocab)}
        self.vovab_size = len(self.vocab)
        print('building...')
        self._build()
        print('trainning...')
        self._train()
        print('trainning Done')
        self.trained_w2v_weight = self.sess.run(self.w2v_weight)
    
    def _build(self):
        with tf.name_scope("input"):
            self.x_input = tf.placeholder(dtype=tf.float32, shape = [None, self.vovab_size], name = "input_x")
            self.y_input = tf.placeholder(dtype=tf.float32, shape = [None, self.vovab_size], name = "input_y")
        with tf.name_scope("fcl"):
            self.w2v_weight = tf.Variable(tf.random_uniform([self.vovab_size, self.w2v_size]),
                                     name = "w2v_weight")
            self.x_present = tf.matmul(self.x_input, self.w2v_weight)
            print('ss', self.x_present.shape)
            self.x_present = tf.reduce_mean(self.x_present, axis = 0)
            print('ss', self.x_present.shape)
            self.x_present = tf.expand_dims(self.x_present, 0)
            print('ss', self.x_present.shape)
        with tf.name_scope("output"):
            self.output_weight = tf.Variable(tf.random_uniform([self.w2v_size, self.vovab_size]),
                                     name = "w2v_weight")
            self.fcl_output = tf.matmul(self.x_present, self.output_weight, name = "output")
        with tf.name_scope("loss"):
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                    logits = self.fcl_output,
                    labels = self.y_input
                )
            )
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
        self.train_step = self.optimizer.minimize(self.loss)
    
    def vec(self, word):
        if(word not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word, self.min_frequency))
            return
        else:
            oh = self._word_onehot(word).reshape(1, -1).astype(np.float32)
            return self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
    
    def _cos(self, vec1, vec2): 
        return np.sum(vec1 * vec2) / (np.sqrt(np.sum(vec1 * vec1)) * np.sqrt(np.sum(vec2 * vec2)))
    
    def sim(self, word1, word2):
        if(word1 not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word1, self.min_frequency))
            return
        elif(word2 not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word2, self.min_frequency))
            return
        else:
            oh1 = self._word_onehot(word1).reshape(1, -1).astype(np.float32)
            oh2 = self._word_onehot(word2).reshape(1, -1).astype(np.float32)
            vec1 = self.sess.run(tf.matmul(oh1, self.w2v_weight))[0]
            vec2 = self.sess.run(tf.matmul(oh2, self.w2v_weight))[0]
            return self._cos(vec1, vec2)
    
    def check_words(self, words):
        for word in words:
            if(word not in self.vocab):
                print('word %s not in vocab maybe frequency < %d' % (word, self.min_frequency))
                return False
        return True
    
    def most_sim(self, pos_words, k, neg_words = []):
        if(False == self.check_words(pos_words) or False == self.check_words(neg_words)):
            return
        else:
            vec = None
            for pw in pos_words:
                oh = self._word_onehot(pw).reshape(1, -1).astype(np.float32)
                if(vec is None):
                    vec = self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
                else:
                    vec += self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
            for nw in neg_words:
                oh = self._word_onehot(nw).reshape(1, -1).astype(np.float32)
                if(vec is None):
                    vec = self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
                else:
                    vec -= self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
            a = vec
            b = self.trained_w2v_weight
            sims = (np.sum(a * b, axis = 1).reshape(-1, 1)) / (np.sqrt(np.sum(b*b, axis = 1).reshape(-1, 1)) * np.sqrt(np.sum(a*a)))
            sims_dict = dict(zip(self.vocab, sims.reshape(1, -1).tolist()[0]))
            #sims_dict = sorted(sims_dict, reverse = True)[1:k+1]
            sims_dict = sorted(sims_dict.items(), key = lambda d:d[1] , reverse = True)
            for tup in sims_dict:
                print(tup)
            return sims_dict
    
    def _word_onehot(self, word):
        if(word == None or word not in self.vocab):
            return self._onehot_encode(-1, self.vovab_size)
        else:
            return self._onehot_encode(self.vocab_index[word], self.vovab_size)
    
    def _extract_window_onehot(self, window, window_side_len):
        mid = self._word_onehot(window[window_side_len])
        sides = []
        for word in window[:window_side_len] + window[window_side_len + 1:]:
            if(word != None):
                sides.append(self._word_onehot(word))
        return mid, sides
    
    def _extract_window(self, line, mid_index, window_side_len):
        window = [None for i in range(2 * window_side_len + 1)]
        window_index = 0
        for index in range(mid_index - window_side_len , mid_index + window_side_len + 1):
            if(index < 0 or index >= len(line)):
                window_index += 1
                continue
            window[window_index] = line[index]
            window_index += 1
        return self._extract_window_onehot(window, window_side_len)
        #print('mid', mid_index, 'mid_word', line[mid_index], 'window', window_side_len)
        #print('line', line)
        #print(window)
    
    def _train(self):
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        print("start trainning model %s" % (self.model_type))
        for epoch in range(self.n_epochs):
            if(self.model_type == "cbow"):
                line_index = 0
                for line in self.words:
                    if(line_index %1000 == 0):
                        print('epoch %d / %d trainning line %d / %d' % (epoch + 1, self.n_epochs, 
                                                                        line_index, len(self.words)))
                    line_index += 1
                    for mid_index, mid_word in enumerate(line):
                        mid, sides = self._extract_window(line, mid_index, self.c_of_cbow)
                        if(0 == len(sides)):
                            continue
                        if(False == np.equal(mid, 0).all() and False == np.equal(sides, 0).all()):
                            batch_x = np.reshape(sides, (len(sides), -1))
                            batch_y = np.reshape(mid, (1, -1))
                        feed_dict = {
                            self.x_input : batch_x,
                            self.y_input : batch_y
                        }
                        #print(feed_dict)
                        self.sess.run(self.train_step, feed_dict = feed_dict)
            elif(self.model_type == "skip-gram"):
                line_index = 0
                for line in self.words:
                    if(line_index %1000 == 0):
                        print('epoch %d / %d trainning line %d / %d' % (epoch + 1, self.n_epochs, line_index, 
                                                                        len(self.words)))
                    line_index += 1
                    for mid_index, mid_word in enumerate(line):
                        mid, sides = self._extract_window(line, mid_index, self.n_of_skgram)
                        if(0 == len(sides)):
                            continue
                        for side in sides:
                            if(False == np.equal(mid, 0).all() and False == np.equal(side, 0).all()):
                                batch_x = np.reshape(mid, (1, -1))
                                batch_y = np.reshape(side, (1, -1))
                                feed_dict = {
                                    self.x_input : batch_x,
                                    self.y_input : batch_y
                                }
                                #print(batch_x.shape, batch_y.shape)
                                self.sess.run(self.train_step, feed_dict = feed_dict)

            else:
                print("train error model type wrong %s" % (self.model_type))
            
            
m_w2v_model = Word2Vec(5, n_epochs=200, learning_rate=0.01, c_of_cbow=2, model_type="skip-gram")
m_w2v_model.fit(['king a','queen a','king b','queen b','king c','queen c','king x',
                            'queen y','man d','woman d','man e','woman e','man f','woman f',
                            'man x','woman y'])
m_w2v_model.most_sim(pos_words=['king', 'woman'], neg_words=['man'], k=2)

ImportError: Traceback (most recent call last):
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/imp.py", line 343, in load_dynamic
    return _l

In [ ]:
cm_w2v_model = Word2Vec(20, n_epochs=500, learning_rate=0.01)
cm_w2v_model.fit(corpus[0:20000])

building...
ss (?, 20)
ss (20,)
ss (1, 20)
trainning...
start trainning model cbow
epoch 1 / 500 trainning line 0 / 20000
epoch 1 / 500 trainning line 1000 / 20000
epoch 1 / 500 trainning line 2000 / 20000
epoch 1 / 500 trainning line 3000 / 20000
epoch 1 / 500 trainning line 4000 / 20000
epoch 1 / 500 trainning line 5000 / 20000
epoch 1 / 500 trainning line 6000 / 20000
epoch 1 / 500 trainning line 7000 / 20000
epoch 1 / 500 trainning line 8000 / 20000
epoch 1 / 500 trainning line 9000 / 20000
epoch 1 / 500 trainning line 10000 / 20000
epoch 1 / 500 trainning line 11000 / 20000
epoch 1 / 500 trainning line 12000 / 20000
epoch 1 / 500 trainning line 13000 / 20000
epoch 1 / 500 trainning line 14000 / 20000
epoch 1 / 500 trainning line 15000 / 20000
epoch 1 / 500 trainning line 16000 / 20000
epoch 1 / 500 trainning line 17000 / 20000
epoch 1 / 500 trainning line 18000 / 20000
epoch 1 / 500 trainning line 19000 / 20000
epoch 2 / 500 trainning line 0 / 20000
epoch 2 / 500 trainning line 10

epoch 10 / 500 trainning line 12000 / 20000
epoch 10 / 500 trainning line 13000 / 20000
epoch 10 / 500 trainning line 14000 / 20000
epoch 10 / 500 trainning line 15000 / 20000
epoch 10 / 500 trainning line 16000 / 20000
epoch 10 / 500 trainning line 17000 / 20000
epoch 10 / 500 trainning line 18000 / 20000
epoch 10 / 500 trainning line 19000 / 20000
epoch 11 / 500 trainning line 0 / 20000
epoch 11 / 500 trainning line 1000 / 20000
epoch 11 / 500 trainning line 2000 / 20000
epoch 11 / 500 trainning line 3000 / 20000
epoch 11 / 500 trainning line 4000 / 20000
epoch 11 / 500 trainning line 5000 / 20000
epoch 11 / 500 trainning line 6000 / 20000
epoch 11 / 500 trainning line 7000 / 20000
epoch 11 / 500 trainning line 8000 / 20000
epoch 11 / 500 trainning line 9000 / 20000
epoch 11 / 500 trainning line 10000 / 20000
epoch 11 / 500 trainning line 11000 / 20000
epoch 11 / 500 trainning line 12000 / 20000
epoch 11 / 500 trainning line 13000 / 20000
epoch 11 / 500 trainning line 14000 / 20000
e

epoch 20 / 500 trainning line 1000 / 20000
epoch 20 / 500 trainning line 2000 / 20000
epoch 20 / 500 trainning line 3000 / 20000
epoch 20 / 500 trainning line 4000 / 20000
epoch 20 / 500 trainning line 5000 / 20000
epoch 20 / 500 trainning line 6000 / 20000
epoch 20 / 500 trainning line 7000 / 20000
epoch 20 / 500 trainning line 8000 / 20000
epoch 20 / 500 trainning line 9000 / 20000
epoch 20 / 500 trainning line 10000 / 20000
epoch 20 / 500 trainning line 11000 / 20000
epoch 20 / 500 trainning line 12000 / 20000
epoch 20 / 500 trainning line 13000 / 20000
epoch 20 / 500 trainning line 14000 / 20000
epoch 20 / 500 trainning line 15000 / 20000
epoch 20 / 500 trainning line 16000 / 20000
epoch 20 / 500 trainning line 17000 / 20000
epoch 20 / 500 trainning line 18000 / 20000
epoch 20 / 500 trainning line 19000 / 20000
epoch 21 / 500 trainning line 0 / 20000
epoch 21 / 500 trainning line 1000 / 20000
epoch 21 / 500 trainning line 2000 / 20000
epoch 21 / 500 trainning line 3000 / 20000
epoc

In [ ]:
cm_w2v_model.most_sim(pos_words=["鸡蛋"], k=100)

In [ ]:
sm_w2v_model = Word2Vec(20, model_type="skip-gram", n_epochs=500, learning_rate=0.01)
sm_w2v_model.fit(corpus[0:20000])


In [ ]:
sm_w2v_model.most_sim(pos_words=["鸡蛋"], k=100)